In [ ]:
import os
from os import path
from astropy.time import Time
from astropy.io import fits, ascii
import astropy.units as u
from astropy.table import Table
from astropy.constants import G

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import h5py
from sqlalchemy import func
from scipy.optimize import root
from scipy.stats import scoreatpercentile
import tqdm

from thejoker import JokerSamples
from thejoker.sampler import JokerParams, TheJoker
from thejoker.plot import plot_rv_curves

from twoface.config import TWOFACE_CACHE_PATH
from twoface.db import (db_connect, AllStar, AllVisit, AllVisitToAllStar, 
                        StarResult, Status, JokerRun, initialize_db)
from twoface.data import APOGEERVData
from twoface.plot import plot_data_orbits, plot_two_panel, plot_phase_fold
from twoface.mass import m2_func
from twoface.samples_analysis import unimodal_P

In [ ]:
samples_file = path.join(TWOFACE_CACHE_PATH, 'apogee-jitter.hdf5')
Session, _ = db_connect(path.join(TWOFACE_CACHE_PATH, 'apogee.sqlite'))
session = Session()

In [ ]:
for i in range(5):
    status = session.query(Status).filter(Status.id == i).limit(1).one()
    n = session.query(AllStar)\
               .join(StarResult, JokerRun, Status)\
               .filter(Status.id == i).count()
    print("Status: {0} ({1}) - {2}".format(status.message, status.id, n))

## TODO: add similar for control sample?